## Install necessary packages

In [32]:
pip install PrettyTable

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


## Create Extract Function (Extract from Etherscan via RESTful API)

In [35]:
#import modules necessary
from prettytable import PrettyTable
from datetime import datetime
import json
import requests
import pyodbc #install pyodbc which is a module which will allow connection of python script to the database using ODBC (Open Database Connectivity) drivers.

#define a function to extract bitcoin data from etherscan via api

def ExtractAPI(input_contract_address):
    # Set API key and ERC-20 token address
    api_key = 'UX7D82IG4G4STQIXZF4SBWNPYNBM7E6B6E'
    contract_address = input_contract_address

    api_endpoint = 'https://api.etherscan.io/api'
    api_params = {
        'module': 'account',
        'action': 'tokentx',
        'contractaddress': contract_address,
        'sort': 'desc',  # Sort transactions in descending order (most recent first)
        'offset': 50,  # Gets the most recent 50 transactions (this is optional)
        'page': 1,  # Get transactions from the first page
        'apikey': api_key
    }

    transactions = []  # Initialize an empty list to store transaction data

    # Paginate through the transaction results until we have xx transactions
    response = requests.get(api_endpoint, params=api_params)
    data = json.loads(response.text)['result']
    transactions.extend(data)  # Add the transactions from this page to our list
    api_params['page'] += 1  # Move to the next page of transactions

    # print(transactions)

    # Convert value to actual value
    for d in transactions:
        d['value'] = float(d['value'])/10**float(d['tokenDecimal'])

    # Sort data by timestamp in descending order
    sorted_data = sorted(transactions, key=lambda k: k['timeStamp'], reverse=True)

    # Convert timestamp in sorted_data to yyyy/mm/dd HH:MM:SS format
    for d in sorted_data:
        d['dateTime'] = datetime.fromtimestamp(int(d['timeStamp'])).strftime('%Y-%m-%dT%H:%M:%S')

    return sorted_data

## Create Transform Function (Tranform and clean JSON data)

In [36]:
def TransformJSON(sorted_data):

    # Transform data as necessary
    # Loop through the data and clean it
    
    transformed_data = []
    for d in sorted_data:
        if float(d['value']) > 100000:
            d['amount_flag'] = 'High'
        if float(d['value']) < 1000:
            d['amount_flag'] = 'Low'  
        else:
            d['amount_flag'] = 'Medium'

        # Remove the timestamp
        d.pop('timeStamp', None)
    
        # Convert 'value' to float (if it's not already)
        d['value'] = float(d['value'])

        # print(transformed_data)

        transformed_data.append(d) #appends the new data to the list

    #table = PrettyTable()  
    #table.field_names = transformed_data[0].keys()  # Set table headers based on keys in the first dictionary

    #for i in transformed_data:
        #table.add_row(i.values()) # Adds new rows to the table by passing the values of the current dictionary 'i' as a list

    return transformed_data #return table - to return pretty table

## Create Load Function (Load to SSMS)

In [37]:
def loadSSMS(input_transformed_data, tokenName):
    # connect to SQL Server database using Windows authentication
    conn=pyodbc.connect(Trusted_Connection='Yes',
                          Driver='{ODBC Driver 17 for SQL Server}',
                          Server='Andre',
                          Database='Test')
    cursor=conn.cursor()
    
    #Create the table in ssms
    create_table_sql='''
    IF NOT EXISTS (SELECT *
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_NAME = 'ERC_Tokens')
    BEGIN
        CREATE TABLE ERC_Tokens ([blockNumber] int,
                            [hash] varchar(MAX),
                            [nonce] int,
                            [blockHash] varchar(MAX),
                            [from] varchar(MAX),
                            [contractAddress] varchar(MAX),
                            [to] varchar(MAX),
                            [value] float,
                            [tokenName] varchar(MAX),
                            [tokenSymbol] varchar(MAX),
                            [tokenDecimal] int,
                            [transactionIndex] int,
                            [gas] float,
                            [gasPrice] nvarchar(MAX),
                            [gasUsed] float,
                            [cumulativeGasUsed] float,
                            [input] varchar(MAX),
                            [confirmations] int,
                            [dateTime] datetime,
                            [amount_flag] varchar(MAX)
                            )
                            
    END
    '''
    cursor.execute(create_table_sql)
    # print('Table has been created')
    conn.commit()
    
    #------
    
    # Define your SQL INSERT statement
    insert_sql = '''
        INSERT INTO ERC_Tokens ([blockNumber],
                            [hash],
                            [nonce],
                            [blockHash],
                            [from],
                            [contractAddress],
                            [to],
                            [value],
                            [tokenName],
                            [tokenSymbol],
                            [tokenDecimal],
                            [transactionIndex],
                            [gas],
                            [gasPrice],
                            [gasUsed],
                            [cumulativeGasUsed],
                            [input],
                            [confirmations],
                            [dateTime],
                            [amount_flag])
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        '''
    
    # Loop through the list of dictionaries and insert data
    for row in input_transformed_data:
        values = (
            row["blockNumber"],      # Assuming blockNumber is an integer
            row["hash"],                  # Assuming hash is a string
            row["nonce"],            # Assuming nonce is an integer
            row["blockHash"],             # Assuming blockHash is a string
            row["from"],                  # Assuming from is a string
            row["contractAddress"],       # Assuming contractAddress is a string
            row["to"],                    # Assuming to is a string
            row["value"],          # Assuming value is a float
            row["tokenName"],             # Assuming tokenName is a string
            row["tokenSymbol"],           # Assuming tokenSymbol is a string
            row["tokenDecimal"],     # Assuming tokenDecimal is an integer
            row["transactionIndex"], # Assuming transactionIndex is an integer
            row["gas"],              # Assuming gas is an integer
            row["gasPrice"],         # Assuming gasPrice is an integer
            row["gasUsed"],          # Assuming gasUsed is an integer
            row["cumulativeGasUsed"], # Assuming cumulativeGasUsed is an integer
            row["input"],                 # Assuming input is a string
            row["confirmations"],    # Assuming confirmations is an integer
            row["dateTime"],              # Assuming dateTime is a string
            row["amount_flag"]            # Assuming amount_flag is a string
        )
    
       
    # Execute the SQL INSERT statement with parameterized values
        cursor.execute(insert_sql, values)
    # Commit the transaction to insert data into the table
    conn.commit()
    print(f'{tokenName} has been successfully inserted')
    
    #------
    
    duplicate_sql = '''
    WITH CTE AS (
            SELECT *,
                   ROW_NUMBER() OVER (PARTITION BY dateTime, hash, value, tokenSymbol ORDER BY (SELECT NULL)) AS rn
            FROM ERC_Tokens
        )
        DELETE FROM CTE WHERE rn > 1;
    '''
    cursor.execute(duplicate_sql)
    # print('Duplicate rows have been deleted')
    conn.commit()
    
    # print(filtered_orders_df.count())
    # Close the cursor and the connection
    cursor.close()
    conn.close()

## Run the code on 10 different tokens

In [38]:
# 1 Shiba inu data stored in 'si_result' - GET SHIBA INU
tokenName = 'SHIBA INU'
si_result=ExtractAPI('0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE')
si_result_transformed=TransformJSON(si_result)
loadSSMS(si_result_transformed, tokenName)

# 2 Tether USD data stored in 'tether_result' - GET TETHER USD
tokenName = 'TETHER USD'
tether_result=ExtractAPI('0xdAC17F958D2ee523a2206206994597C13D831ec7')
tether_result_transformed=TransformJSON(tether_result)
loadSSMS(tether_result_transformed, tokenName)

# 3 BNB data stored in 'BNB_result' - GET BNB
tokenName = 'BNB'
BNB_result=ExtractAPI('0xB8c77482e45F1F44dE1745F52C74426C631bDD52')
BNB_result_transformed=TransformJSON(BNB_result)
loadSSMS(BNB_result_transformed, tokenName)

# 4 stETH data stored in 'stETH_result' - GET stETH
tokenName = 'stETH'
stETH_result=ExtractAPI('0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84')
stETH_result_transformed=TransformJSON(stETH_result)
loadSSMS(stETH_result_transformed, tokenName)

# 5 TRON data stored in 'TRON_result' - GET TRON
tokenName = 'TRON'
TRON_result=ExtractAPI('0x50327c6c5a14DCaDE707ABad2E27eB517df87AB5')
TRON_result_transformed=TransformJSON(TRON_result)
loadSSMS(TRON_result_transformed, tokenName)

# 6 Theta data stored in 'Theta_result' - GET Theta
tokenName = 'Theta'
Theta_result=ExtractAPI('0x3883f5e181fccaF8410FA61e12b59BAd963fb645')
Theta_result_transformed=TransformJSON(Theta_result)
loadSSMS(Theta_result_transformed, tokenName)

# 7 Matic data stored in 'Matic_result' - GET Matic
tokenName = 'Matic'
Matic_result=ExtractAPI('0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0')
Matic_result_transformed=TransformJSON(Matic_result)
loadSSMS(Matic_result_transformed, tokenName)

# 8 ChainLink data stored in 'ChainLink_result' - GET ChainLink
tokenName = 'ChainLink'
ChainLink_result=ExtractAPI('0x514910771AF9Ca656af840dff83E8264EcF986CA')
ChainLink_result_transformed=TransformJSON(ChainLink_result)
loadSSMS(ChainLink_result_transformed, tokenName)

# 9 Dai Stablecoin data stored in 'Dai_result' - GET Dai
tokenName = 'Dai'
Dai_result=ExtractAPI('0x6B175474E89094C44Da98b954EedeAC495271d0F')
Dai_result_transformed=TransformJSON(Dai_result)
loadSSMS(Dai_result_transformed, tokenName)

# 10 Uniswap Stablecoin data stored in 'Uni' - GET Uni
tokenName = 'Uniswap'
Uni_result=ExtractAPI('0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984')
Uni_result_transformed=TransformJSON(Uni_result)
loadSSMS(Uni_result_transformed, tokenName)

SHIBA INU has been successfully inserted
TETHER USD has been successfully inserted
BNB has been successfully inserted
stETH has been successfully inserted
TRON has been successfully inserted
Theta has been successfully inserted
Matic has been successfully inserted
ChainLink has been successfully inserted
Dai has been successfully inserted
Uniswap has been successfully inserted
